In [1]:
import math
from top2vec import Top2Vec
import numpy as np
import pandas as pd
import umap
import pandas as pd
from csv import writer
#import umap.plot
import matplotlib.pyplot as plt
import csv
import pickle

/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/charlottepanuskova/anaconda3/lib/python3.11/site-packages/uma

In [3]:
path = "data/lem/all"
file_name =  "jerome_synv9_id_no_names"# "jerome_synv9_id" #_NOV_MEM_COL_deep_learn
file_path = path+"/"+file_name + ".txt"
chunk = 5000
divide = "whole books"
txtype_select = ['NOV: próza', 'MEM: memoáry, autobiografie', 'COL: kratší próza']

In [19]:

def load_books_chunks_from_document(CONST, file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    books = []
    books_info = []
    start_tag = '<doc title="'
    end_tag = '</doc>'
    start_index = 0

    while True:
        book_start = content.find(start_tag, start_index)
        if book_start == -1:
            break

        book_end = content.find(end_tag, book_start)
        if book_end == -1:
            break

        book_text = content[book_start:book_end + len(end_tag)]
        book_info = book_text.strip()[5:book_text.index('>') + 1]  # Remove '<doc' and '</doc>'

        book_info_list = book_info.split('" ')
        book_info_dict = {}

        for item in book_info_list:
            key, value = item.split('=')
            book_info_dict[key.strip()] = value.strip('"')

        book_content = book_text[book_text.index('>') + 1:-len(end_tag)].strip()
        book_content = book_content.split(' ')
        length = len(book_content)
        for i in range(math.ceil(length/CONST)):
            i = i*CONST
            end = i+CONST if i+CONST < length-1 else length-1
            books.append(" ".join(book_content[i:end]))
            books_info.append(book_info_dict)

        start_index = book_end + len(end_tag)

    return books, books_info


def load_books_from_document(file_path, txtype_select):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    books = []
    books_info = []
    start_tag = '<doc title="'
    end_tag = '</doc>'
    start_index = 0

    while True:
        book_start = content.find(start_tag, start_index)
        if book_start == -1:
            break

        book_end = content.find(end_tag, book_start)
        if book_end == -1:
            break

        book_text = content[book_start:book_end + len(end_tag)]
        book_info = book_text.strip()[5:book_text.index('>') + 1]  # Remove '<doc' and '</doc>'  # Remove '<doc' and '</doc>'

        
        if txtype_select != [] and any([txtype in book_info for txtype in txtype_select]):
            book_info_list = book_info.split('" ') 
            book_info_dict = {}

            for item in book_info_list:
                key, value = item.split('=')
                book_info_dict[key.strip()] = value.strip('"')

            book_content = book_text[book_text.index('>') + 1:-len(end_tag)].strip()        
            books.append(book_content)

            books_info.append(book_info_dict)

        start_index = book_end + len(end_tag)

    return books, books_info



def load_books_blocks_from_document(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    books = []
    books_info = []
    start_tag = '<doc title="'
    end_tag = '</doc>'
    start_index = 0

    while True:
        book_start = content.find(start_tag, start_index)
        if book_start == -1:
            break

        book_end = content.find(end_tag, book_start)
        if book_end == -1:
            break

        book_text = content[book_start:book_end + len(end_tag)]
        book_info = book_text.strip()[5:book_text.index('>') + 1]  # Remove '<doc' and '</doc>'  # Remove '<doc' and '</doc>'

        book_info_list = book_info.split('" ')
        book_info_dict = {}

        for item in book_info_list:
            key, value = item.split('=')
            book_info_dict[key.strip()] = value.strip('"')

        book_content = book_text[book_text.index('>') + 1:-len(end_tag)].strip()
        book_content = book_content.split('\n')
        for book_block in book_content:
            books.append(book_block)
            books_info.append(book_info_dict)

        start_index = book_end + len(end_tag)

    return books, books_info


def load_books_from_document_without_residual_ner(file_path, txtype_select, df_dict_ner):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    books = []
    books_info = []
    start_tag = '<doc title="'
    end_tag = '</doc>'
    start_index = 0

    while True:
        book_start = content.find(start_tag, start_index)
        if book_start == -1:
            break

        book_end = content.find(end_tag, book_start)
        if book_end == -1:
            break

        book_text = content[book_start:book_end + len(end_tag)]
        book_info = book_text.strip()[5:book_text.index('>') + 1]  # Remove '<doc' and '</doc>'  # Remove '<doc' and '</doc>'

        
        if txtype_select != [] and any([txtype in book_info for txtype in txtype_select]):
            book_info_list = book_info.split('" ') 
            book_info_dict = {}

            for item in book_info_list:
                key, value = item.split('=')
                book_info_dict[key.strip()] = value.strip('"')
                

            book_content = book_text[book_text.index('>') + 1:-len(end_tag)].strip().split(' ')
            dict_key = (book_info_dict['title'],book_info_dict['author'] )
            if dict_key in df_dict_ner.keys():
                book_content = ' '.join([i for i in book_content if i not in df_dict_ner[(book_info_dict['title'],book_info_dict['author'] )]])  
            else:
                book_content = ' '.join(book_content)    
            books.append(book_content)


            books_info.append(book_info_dict)

        start_index = book_end + len(end_tag)

    return books, books_info



def load_all_names(df_ner):
    df_dict_ner={}
    for _, df in df_ner.groupby(['title','author']):
        title = df.title.unique()[0]
        author = df.author.unique()[0]
        # get rid of strings that are too short 
        df_dict_ner[(title, author)] = set(filter(lambda x: len(x)>2,df['name entity'].unique().tolist()))
        
    return df_dict_ner    




  

In [2]:
with open('names_frame.obj', 'rb') as f:
    names_frame = pickle.load(f)
    df_ner = pd.DataFrame(names_frame)
    
df_dict_ner = load_all_names(df_ner)  
if divide == "chunk":
    books, books_info = load_books_chunks_from_document(chunk, file_path)
elif divide == "block":    
    books, books_info = load_books_blocks_from_document(file_path)
else:
    books, books_info = load_books_from_document_without_residual_ner(file_path, txtype_select, df_dict_ner)   

 

NameError: name 'load_all_names' is not defined

In [3]:
names_frame

{'title': ['Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'P

In [22]:
# Create the Top2Vec model
top2vec_model = Top2Vec(books, embedding_model="doc2vec", speed='fast-learn', workers = 8)

top2vec_model.save('models/top2vec_jerome_synv9_fast_learn_no_names_without_residuals')

2024-04-06 16:51:31,583 - top2vec - INFO - Pre-processing documents for training
2024-04-06 16:51:57,437 - top2vec - INFO - Creating joint document/word embedding
2024-04-06 17:05:28,051 - top2vec - INFO - Creating lower dimension embedding of documents
2024-04-06 17:05:45,146 - top2vec - INFO - Finding dense areas of documents
2024-04-06 17:05:45,160 - top2vec - INFO - Finding topics


In [18]:
df_ner

,title,author,type,name entity
0,Planeta lykantů,"Garen, Jean-Pierre",ps,Edwinovým
1,Planeta lykantů,"Garen, Jean-Pierre",pf,Marka
2,Planeta lykantů,"Garen, Jean-Pierre",pf,Marka
3,Planeta lykantů,"Garen, Jean-Pierre",pf,Peggy
4,Planeta lykantů,"Garen, Jean-Pierre",ps,Stone
...,...,...,...,...
63181643,Metodický pokyn pro výkon inspekce práce,X,oe,Sb
63181644,Metodický pokyn pro výkon inspekce práce,X,o_,c
63181645,Metodický pokyn pro výkon inspekce práce,X,o_,c
63181646,Metodický pokyn pro výkon inspekce práce,X,ni,3


{'title': ['Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'Planeta lykantů',
  'P